In [12]:
%matplotlib inline

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString, shape
from shapely.geometry.polygon import Polygon
import requests
import pprint
import json
import numpy as np
import pyproj
import os
import folium
import time 
import math
import warnings
warnings.filterwarnings('ignore')

In [13]:
bsGJ= pd.read_csv("./busstop_GJ.csv")
bsGJ

,Unnamed: 0,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,geometry,sgg_oid,jibun,pnu,index_right,ADM_DR_CD,ADM_DR_NM
0,5018,274000018,잠실대교북단(경유),일반차로,78449,127.088565,37.528817,POINT (127.088565 37.528817),96951,753-3도,1121510500107530003,77,1105065,자양2동
1,2942,277102301,잠실대교북단(경유),일반차로,78353,127.090492,37.529313,POINT (127.090492 37.529313),96289,701제,1121510500107010000,77,1105065,자양2동
2,5017,274000019,잠실대교북단(경유),일반차로,78450,127.090479,37.529354,POINT (127.090479 37.529354),96289,701제,1121510500107010000,77,1105065,자양2동
3,69003,104900116,행운슈퍼앞,마을버스,5572,127.087010,37.532950,POINT (127.08701 37.53295),94599,682-15대,1121510500106820015,77,1105065,자양2동
4,69005,104900114,성자초등학교앞,마을버스,5562,127.082256,37.531300,POINT (127.082256 37.5313),95777,652-31대,1121510500106520031,77,1105065,자양2동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,69205,104000127,영동대교북단,일반차로,5220,127.061162,37.536568,POINT (127.061162 37.536568),98481,235-3전,1121510500102350003,79,1105067,자양4동
334,69207,104000125,건대로데오거리입구,일반차로,5218,127.069926,37.538919,POINT (127.069926 37.538919),247655,2-65 도,1121510500100020065,79,1105067,자양4동
335,69208,104000124,건대입구역6번출구,일반차로,5217,127.067279,37.540827,POINT (127.067279 37.540827),97155,8-24대,1121510500100080024,79,1105067,자양4동
336,69209,104000123,노룬산시장,일반차로,5216,127.062691,37.539503,POINT (127.062691 37.539503),97503,250-10대,1121510500102500010,79,1105067,자양4동


In [14]:
bsGJ.drop(['index_right','Unnamed: 0','geometry'], axis=1,inplace=True)
bsGJ

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,sgg_oid,jibun,pnu,ADM_DR_CD,ADM_DR_NM
0,274000018,잠실대교북단(경유),일반차로,78449,127.088565,37.528817,96951,753-3도,1121510500107530003,1105065,자양2동
1,277102301,잠실대교북단(경유),일반차로,78353,127.090492,37.529313,96289,701제,1121510500107010000,1105065,자양2동
2,274000019,잠실대교북단(경유),일반차로,78450,127.090479,37.529354,96289,701제,1121510500107010000,1105065,자양2동
3,104900116,행운슈퍼앞,마을버스,5572,127.087010,37.532950,94599,682-15대,1121510500106820015,1105065,자양2동
4,104900114,성자초등학교앞,마을버스,5562,127.082256,37.531300,95777,652-31대,1121510500106520031,1105065,자양2동
...,...,...,...,...,...,...,...,...,...,...,...
333,104000127,영동대교북단,일반차로,5220,127.061162,37.536568,98481,235-3전,1121510500102350003,1105067,자양4동
334,104000125,건대로데오거리입구,일반차로,5218,127.069926,37.538919,247655,2-65 도,1121510500100020065,1105067,자양4동
335,104000124,건대입구역6번출구,일반차로,5217,127.067279,37.540827,97155,8-24대,1121510500100080024,1105067,자양4동
336,104000123,노룬산시장,일반차로,5216,127.062691,37.539503,97503,250-10대,1121510500102500010,1105067,자양4동


In [15]:
bsGJ['경도'] = bsGJ['경도'].astype(float)
bsGJ['위도'] = bsGJ['위도'].astype(float)
bsGJ['geometry'] = bsGJ.apply(lambda row : Point([row['위도'], row['경도']]), axis=1)
bsGJ = gpd.GeoDataFrame(bsGJ, geometry='geometry')
bsGJ

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,sgg_oid,jibun,pnu,ADM_DR_CD,ADM_DR_NM,geometry
0,274000018,잠실대교북단(경유),일반차로,78449,127.088565,37.528817,96951,753-3도,1121510500107530003,1105065,자양2동,POINT (127.08857 37.52882)
1,277102301,잠실대교북단(경유),일반차로,78353,127.090492,37.529313,96289,701제,1121510500107010000,1105065,자양2동,POINT (127.09049 37.52931)
2,274000019,잠실대교북단(경유),일반차로,78450,127.090479,37.529354,96289,701제,1121510500107010000,1105065,자양2동,POINT (127.09048 37.52935)
3,104900116,행운슈퍼앞,마을버스,5572,127.087010,37.532950,94599,682-15대,1121510500106820015,1105065,자양2동,POINT (127.08701 37.53295)
4,104900114,성자초등학교앞,마을버스,5562,127.082256,37.531300,95777,652-31대,1121510500106520031,1105065,자양2동,POINT (127.08226 37.53130)
...,...,...,...,...,...,...,...,...,...,...,...,...
333,104000127,영동대교북단,일반차로,5220,127.061162,37.536568,98481,235-3전,1121510500102350003,1105067,자양4동,POINT (127.06116 37.53657)
334,104000125,건대로데오거리입구,일반차로,5218,127.069926,37.538919,247655,2-65 도,1121510500100020065,1105067,자양4동,POINT (127.06993 37.53892)
335,104000124,건대입구역6번출구,일반차로,5217,127.067279,37.540827,97155,8-24대,1121510500100080024,1105067,자양4동,POINT (127.06728 37.54083)
336,104000123,노룬산시장,일반차로,5216,127.062691,37.539503,97503,250-10대,1121510500102500010,1105067,자양4동,POINT (127.06269 37.53950)


In [16]:
bsGJ.set_crs(epsg=4326, inplace=True)
bsGJ = bsGJ.to_crs({'init':'epsg:5179'})
bsGJ

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,sgg_oid,jibun,pnu,ADM_DR_CD,ADM_DR_NM,geometry
0,274000018,잠실대교북단(경유),일반차로,78449,127.088565,37.528817,96951,753-3도,1121510500107530003,1105065,자양2동,POINT (963647.129 1947802.874)
1,277102301,잠실대교북단(경유),일반차로,78353,127.090492,37.529313,96289,701제,1121510500107010000,1105065,자양2동,POINT (963817.632 1947857.159)
2,274000019,잠실대교북단(경유),일반차로,78450,127.090479,37.529354,96289,701제,1121510500107010000,1105065,자양2동,POINT (963816.503 1947861.713)
3,104900116,행운슈퍼앞,마을버스,5572,127.087010,37.532950,94599,682-15대,1121510500106820015,1105065,자양2동,POINT (963511.748 1948262.008)
4,104900114,성자초등학교앞,마을버스,5562,127.082256,37.531300,95777,652-31대,1121510500106520031,1105065,자양2동,POINT (963090.910 1948080.805)
...,...,...,...,...,...,...,...,...,...,...,...,...
333,104000127,영동대교북단,일반차로,5220,127.061162,37.536568,98481,235-3전,1121510500102350003,1105067,자양4동,POINT (961229.902 1948673.746)
334,104000125,건대로데오거리입구,일반차로,5218,127.069926,37.538919,247655,2-65 도,1121510500100020065,1105067,자양4동,POINT (962005.373 1948930.999)
335,104000124,건대입구역6번출구,일반차로,5217,127.067279,37.540827,97155,8-24대,1121510500100080024,1105067,자양4동,POINT (961772.499 1949143.753)
336,104000123,노룬산시장,일반차로,5216,127.062691,37.539503,97503,250-10대,1121510500102500010,1105067,자양4동,POINT (961366.500 1948998.738)


In [17]:
bsGJ['x'] = bsGJ['geometry'].x
bsGJ['y'] = bsGJ['geometry'].y
bsGJ

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,sgg_oid,jibun,pnu,ADM_DR_CD,ADM_DR_NM,geometry,x,y
0,274000018,잠실대교북단(경유),일반차로,78449,127.088565,37.528817,96951,753-3도,1121510500107530003,1105065,자양2동,POINT (963647.129 1947802.874),963647.128824,1.947803e+06
1,277102301,잠실대교북단(경유),일반차로,78353,127.090492,37.529313,96289,701제,1121510500107010000,1105065,자양2동,POINT (963817.632 1947857.159),963817.631746,1.947857e+06
2,274000019,잠실대교북단(경유),일반차로,78450,127.090479,37.529354,96289,701제,1121510500107010000,1105065,자양2동,POINT (963816.503 1947861.713),963816.502922,1.947862e+06
3,104900116,행운슈퍼앞,마을버스,5572,127.087010,37.532950,94599,682-15대,1121510500106820015,1105065,자양2동,POINT (963511.748 1948262.008),963511.747643,1.948262e+06
4,104900114,성자초등학교앞,마을버스,5562,127.082256,37.531300,95777,652-31대,1121510500106520031,1105065,자양2동,POINT (963090.910 1948080.805),963090.909966,1.948081e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,104000127,영동대교북단,일반차로,5220,127.061162,37.536568,98481,235-3전,1121510500102350003,1105067,자양4동,POINT (961229.902 1948673.746),961229.901959,1.948674e+06
334,104000125,건대로데오거리입구,일반차로,5218,127.069926,37.538919,247655,2-65 도,1121510500100020065,1105067,자양4동,POINT (962005.373 1948930.999),962005.373337,1.948931e+06
335,104000124,건대입구역6번출구,일반차로,5217,127.067279,37.540827,97155,8-24대,1121510500100080024,1105067,자양4동,POINT (961772.499 1949143.753),961772.498705,1.949144e+06
336,104000123,노룬산시장,일반차로,5216,127.062691,37.539503,97503,250-10대,1121510500102500010,1105067,자양4동,POINT (961366.500 1948998.738),961366.499767,1.948999e+06


In [18]:
grid_data = pd.read_csv("./clustered_grid.csv")
grid_data

,Unnamed: 0,x_index,y_index,격자 당 경로당 수,geometry,행정동,격자 당 어린이 보육시설 수,유아비율(%),노인비율(%),kmeans_label
0,0,0,22,0,"POLYGON ((127.0561797939232 37.53325271346528,...",자양4동,0,2.839451,20.913757,0
1,1,0,23,0,POLYGON ((127.05619047599266 37.53145004424569...,자양4동,0,2.839451,20.913757,0
2,2,0,24,0,POLYGON ((127.05620115711046 37.52964737447184...,자양4동,0,2.839451,20.913757,0
3,3,0,25,0,POLYGON ((127.05621183727676 37.52784470414376...,자양4동,0,2.839451,20.913757,0
4,4,1,20,0,POLYGON ((127.05842217001221 37.53686653680621...,자양4동,0,2.839451,20.913757,0
...,...,...,...,...,...,...,...,...,...,...
478,478,25,8,0,POLYGON ((127.11264035105809 37.55868932524784...,광장동,0,5.025170,11.665886,3
479,479,25,9,0,POLYGON ((127.11264968278773 37.55688665089398...,광장동,0,5.025170,11.665886,3
480,480,25,10,0,POLYGON ((127.11265901368601 37.55508397598633...,광장동,0,5.025170,11.665886,3
481,481,25,11,0,POLYGON ((127.11266834375294 37.55328130052497...,광장동,0,5.025170,11.665886,3


In [19]:
grid_data.drop(['Unnamed: 0','geometry'], axis=1,inplace=True)
grid_data

,x_index,y_index,격자 당 경로당 수,행정동,격자 당 어린이 보육시설 수,유아비율(%),노인비율(%),kmeans_label
0,0,22,0,자양4동,0,2.839451,20.913757,0
1,0,23,0,자양4동,0,2.839451,20.913757,0
2,0,24,0,자양4동,0,2.839451,20.913757,0
3,0,25,0,자양4동,0,2.839451,20.913757,0
4,1,20,0,자양4동,0,2.839451,20.913757,0
...,...,...,...,...,...,...,...,...
478,25,8,0,광장동,0,5.025170,11.665886,3
479,25,9,0,광장동,0,5.025170,11.665886,3
480,25,10,0,광장동,0,5.025170,11.665886,3
481,25,11,0,광장동,0,5.025170,11.665886,3


In [20]:
grid_data2= pd.read_csv("./grid_data2.csv")
grid_data2 = grid_data2.loc[:,['x_index','y_index','중심점','geometry','x','y']]
grid_data2

,x_index,y_index,중심점,geometry,x,y
0,0,22,POINT (960888.0000000005 1948207.9999999998),POLYGON ((960788.0000000006 1948307.9999999998...,960888.0,1948208.0
1,0,23,POINT (960887.9999999998 1948008.0000000002),"POLYGON ((960788.0000000003 1948108, 960988.00...",960888.0,1948008.0
2,0,24,POINT (960887.9999999995 1947808),"POLYGON ((960787.9999999991 1947908.000000001,...",960888.0,1947808.0
3,0,25,POINT (960887.9999999999 1947607.9999999998),POLYGON ((960788.0000000001 1947707.9999999998...,960888.0,1947608.0
4,1,20,POINT (961087.9999999992 1948608.0000000002),"POLYGON ((960987.9999999992 1948708, 961187.99...",961088.0,1948608.0
...,...,...,...,...,...,...
478,25,8,POINT (965888.0000000001 1951007.9999999998),"POLYGON ((965788.000000001 1951108, 965987.999...",965888.0,1951008.0
479,25,9,POINT (965888.0000000003 1950807.9999999998),POLYGON ((965787.9999999993 1950907.9999999995...,965888.0,1950808.0
480,25,10,POINT (965888.0000000001 1950608),"POLYGON ((965788.0000000003 1950707.999999999,...",965888.0,1950608.0
481,25,11,POINT (965887.9999999997 1950408.0000000002),"POLYGON ((965787.9999999995 1950508, 965987.99...",965888.0,1950408.0


In [21]:
grid_data3 = pd.merge(grid_data,grid_data2)
grid_data3 

,x_index,y_index,격자 당 경로당 수,행정동,격자 당 어린이 보육시설 수,유아비율(%),노인비율(%),kmeans_label,중심점,geometry,x,y
0,0,22,0,자양4동,0,2.839451,20.913757,0,POINT (960888.0000000005 1948207.9999999998),POLYGON ((960788.0000000006 1948307.9999999998...,960888.0,1948208.0
1,0,23,0,자양4동,0,2.839451,20.913757,0,POINT (960887.9999999998 1948008.0000000002),"POLYGON ((960788.0000000003 1948108, 960988.00...",960888.0,1948008.0
2,0,24,0,자양4동,0,2.839451,20.913757,0,POINT (960887.9999999995 1947808),"POLYGON ((960787.9999999991 1947908.000000001,...",960888.0,1947808.0
3,0,25,0,자양4동,0,2.839451,20.913757,0,POINT (960887.9999999999 1947607.9999999998),POLYGON ((960788.0000000001 1947707.9999999998...,960888.0,1947608.0
4,1,20,0,자양4동,0,2.839451,20.913757,0,POINT (961087.9999999992 1948608.0000000002),"POLYGON ((960987.9999999992 1948708, 961187.99...",961088.0,1948608.0
...,...,...,...,...,...,...,...,...,...,...,...,...
478,25,8,0,광장동,0,5.025170,11.665886,3,POINT (965888.0000000001 1951007.9999999998),"POLYGON ((965788.000000001 1951108, 965987.999...",965888.0,1951008.0
479,25,9,0,광장동,0,5.025170,11.665886,3,POINT (965888.0000000003 1950807.9999999998),POLYGON ((965787.9999999993 1950907.9999999995...,965888.0,1950808.0
480,25,10,0,광장동,0,5.025170,11.665886,3,POINT (965888.0000000001 1950608),"POLYGON ((965788.0000000003 1950707.999999999,...",965888.0,1950608.0
481,25,11,0,광장동,0,5.025170,11.665886,3,POINT (965887.9999999997 1950408.0000000002),"POLYGON ((965787.9999999995 1950508, 965987.99...",965888.0,1950408.0


In [22]:
grid_data3.to_csv('./grid_result.csv')

In [23]:
bsGJ.to_csv('./bsGJ.csv')